## Version Description:
 * In this version - we predict just the __R600 Hole Diameter__ (`R600_HD`)in TZ6 using [Bayesian Neural Network with MC DropOut](https://github.com/valyome/Neural-Networks-with-MC-Dropout/). 
 * Artificial data points synthesised from FDDN are used to train the ML model
 * The input features are `TZ6_Flow`,`MIXP`,`AMBP`,`AMBT` and the TZ6 CLR Restrictors.

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.6g" % x)) 
from functools import reduce

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
plt.style.use('seaborn-whitegrid')

### Loading Data

In [5]:
input_features = ['TZ_Flow','MIXP','AMBP','AMBT','R610_HS1','R611_HS1','R612_HS1','R613_HS1']
usecols = ['HoV']+input_features+['R600_HD']

In [8]:
FDDN_df = pd.read_csv('../../data/Simulation_dataset.csv', usecols = usecols)[usecols]
FDDN_df.head()

,HoV,TZ_Flow,MIXP,AMBP,AMBT,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R600_HD
0,FDDN_ASD0,720.292743,1800,102926,297.15,131,136,120,120,146
1,FDDN_ASD1,871.403000,2600,102426,299.15,131,136,120,120,146
2,FDDN_ASD2,985.488429,3400,103127,294.15,131,136,120,120,146
3,FDDN_ASD3,722.824643,1800,101825,295.15,131,136,120,120,146
4,FDDN_ASD4,875.039489,2600,100523,295.15,131,136,120,120,146


In [6]:
LTR_df = pd.read_csv('../../data/LTR_dataset.csv', usecols = usecols)[usecols]
LTR_df.head()

,HoV,TZ_Flow,MIXP,AMBP,AMBT,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R600_HD
0,A1,887.134249,2600.0000,101401.6000,299.386667,131,136,120,120,148
1,A2,886.764050,2600.0000,101576.3000,298.448667,131,136,120,120,149
2,A3,926.224856,2606.1928,102136.6035,297.109024,131,136,114,120,152
3,A4,915.139474,2599.8998,103195.6642,295.060027,131,136,120,120,154
4,A5,891.635528,2600.0000,102856.2000,294.755833,145,153,130,130,148


### Train BNN with FDDN (Simulation) data:

In [7]:
# Extract pandas dataframe values to numpy array
train_data = FDDN_df[input_features+['R600_HD']].values
test_data  = LTR_df[input_features+['R600_HD']].values
print("Number of training samples:", len(train_data))
print("\nNumber of testing samples:", len(test_data))

# Separate arrays into input and output components (predictors and response variables)
x_train, y_train  = train_data[:,:-1] ,train_data[:,-1:]
x_test, y_test = test_data[:,:-1] ,test_data[:,-1:]

Number of training samples: 1080

Number of testing samples: 34


In [8]:
print('X_train Shape:',x_train.shape,'Y_Train Shape:', y_train.shape)
print('X_test Shape:',x_test.shape,'Y_Test Shape:', y_test.shape)

X_train Shape: (1080, 8) Y_Train Shape: (1080, 1)
X_test Shape: (34, 8) Y_Test Shape: (34, 1)


### Bayesian Neural Network using MC DropOut

In [9]:
import BNN_MonteCarlo_Dropout

Using TensorFlow backend.


In [10]:
%%time
# Train the BNN
bnn = BNN_MonteCarlo_Dropout.net(x_train, y_train, n_epochs=2000, n_hidden=[24,12], normalize=True)

Wall time: 20.6 s


In [11]:
%%time
# Test on ~25% of the data.
rmse, Yt_hat, MC_pred, predictive_variance = bnn.predict(x_test, y_test)

34/34 [==============================] - 0s 3ms/step
Standard rmse 1.500746
MC rmse 1.480097
test_ll -4.784234
Wall time: 653 ms


In [12]:
print('Shape of Predictions:',Yt_hat.shape)

Shape of Predictions: (1000, 1, 34, 1)


In [13]:
Yt_hat.shape[2]

34

In [14]:
y_preds = np.zeros((Yt_hat.shape[2], Yt_hat.shape[0])) # empty array to be populated
y_means = [] # save mean for each predicted point
y_std=[] # save standard dev for each predicted
y_mins = [] 
y_maxes = []

for j in range(Yt_hat.shape[2]):
    for i in range(Yt_hat.shape[0]):
        y_preds[j][i] = Yt_hat[i][0][j] # convert the predictions into a more readable format
    y_means.append(y_preds[j].mean()) # get the mean for each prediction
    y_std.append(y_preds[j].std()) # get the standard deviation
    y_mins.append(np.amin(y_preds[j],axis = 0)) # get the min value in the array
    y_maxes.append(np.amax(y_preds[j],axis = 0)) # get the min value in the array

In [15]:
print('Length of Y_Predictions:',len(y_preds))
print('Nr. of Predictions by BNN per test point:',len(y_preds[0]))

Length of Y_Predictions: 34
Nr. of Predictions by BNN per test point: 1000


In [16]:
print(y_means)

[147.4601276550293, 147.70334313964844, 152.57540802001952, 151.91761940002442, 147.91444717407228, 148.9410216064453, 148.2716453704834, 151.89613014221192, 151.36993754577637, 148.67149407958985, 150.26849583435057, 153.0716208343506, 149.38465394592285, 148.30811764526368, 147.75713177490235, 149.17297521972657, 146.91343643188478, 146.88077880859376, 147.59547434997558, 147.06445600891112, 149.33490144348144, 147.83673170471192, 154.03554676818848, 152.97437602233887, 153.20926568603517, 147.9585760498047, 148.3189882659912, 152.69357565307618, 148.18679014587403, 149.37595835876465, 151.8161474609375, 154.21317196655272, 146.83084317016602, 148.1894887237549]


In [17]:
print(y_std)

[1.2251693934428505, 1.280517180511695, 0.4940833742102587, 0.5667916971035993, 1.1979694445093083, 0.9955768522088536, 1.200338318865099, 0.4963598073132157, 0.5554825358994416, 1.1530615069897006, 0.7522829615360521, 0.5030924601050561, 0.9133981888490618, 1.203550030255305, 1.4389549463964886, 0.8572004099580494, 1.5953261570756527, 1.4989899327896485, 1.1678809611360896, 1.2194114888531242, 0.8743001199335282, 1.2967640099435496, 0.5903768260597645, 0.6920425994390283, 0.4884576452972184, 1.5430047092841175, 1.0081409235078016, 0.4734975456434873, 1.1473929663898084, 0.8888433203246087, 0.5473348915869954, 0.6303427163373612, 1.4639575411600438, 1.1741402515353085]


In [18]:
print(y_mins)

[146.2541046142578, 146.3826446533203, 150.5880126953125, 149.40487670898438, 146.54774475097656, 147.13233947753906, 146.67941284179688, 149.5604248046875, 148.89779663085938, 146.81509399414062, 147.83029174804688, 150.17367553710938, 147.24147033691406, 146.63722229003906, 146.41897583007812, 147.2540740966797, 145.4420623779297, 145.70062255859375, 146.44488525390625, 145.808837890625, 147.37521362304688, 146.4114990234375, 151.65072631835938, 149.972412109375, 150.99147033691406, 146.28172302246094, 146.72467041015625, 150.524169921875, 146.40087890625, 147.4194793701172, 149.26048278808594, 151.73191833496094, 145.3487091064453, 146.64251708984375]


In [19]:
print(y_maxes)

[152.6996307373047, 154.04367065429688, 155.47113037109375, 154.66783142089844, 153.07472229003906, 154.57901000976562, 153.5592803955078, 155.25086975097656, 154.65223693847656, 154.0426483154297, 153.96412658691406, 155.1548614501953, 154.00306701660156, 154.0738525390625, 154.98980712890625, 153.09811401367188, 153.8497772216797, 154.8241424560547, 152.35153198242188, 153.23812866210938, 153.2869415283203, 153.816162109375, 157.08201599121094, 156.97067260742188, 155.4565887451172, 155.16107177734375, 153.35948181152344, 155.7735595703125, 153.0620880126953, 153.6128387451172, 155.45001220703125, 157.659912109375, 153.31503295898438, 153.0906982421875]


In [20]:
y_true = LTR_df['R600_HD'].values
HoV_test_data = LTR_df['HoV'].values

In [21]:
print('No. of Test HoVs:',len(HoV_test_data),'\nHoVs of Test Points:',HoV_test_data)
print('Test Points-Org Y:', y_true)

No. of Test HoVs: 34 
HoVs of Test Points: ['A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'D1' 'D2' 'E1' 'E2' 'F1' 'F2' 'F3' 'H1' 'I1' 'J1' 'M1' 'M2' 'P1' 'Q1' 'Q2' 'S1' 'S2' 'S3' 'S4' 'S5' 'T1' 'T2']
Test Points-Org Y: [148 149 152 154 148 148 148 153 152 150 152 154 151 148 148 148 146 146 148 148 152 148 154 154 154 148 148 155 148 150 150 160 148 148]


### Plot BNN Predictions with Mean and Error Bars

In [22]:
from bokeh.models import HoverTool,ColumnDataSource,Label,Range1d
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.models.markers import CircleX
output_notebook()

Loading BokehJS ...

In [23]:
# create the coordinates for the errorbars
y_xs = []
ys_std = []
y_ys = []

for x, y_mean,std,min_dia, max_dia in zip(HoV_test_data,y_means,y_std,y_mins,y_maxes):
    y_xs.append((x, x))
    ys_std.append((y_mean - std, y_mean + std))
    y_ys.append((y_mean - (y_mean - min_dia), y_mean + (max_dia - y_mean)))

In [24]:
# plot the points
f = figure(x_range=HoV_test_data, title='BNN MC DropOut - Dia Predictions with Confidence Interval', width=1400)
dot_hover_tooltips = HoverTool(names=['DiaPoints'],tooltips=[("HoV","@x"),("Dia", "@y mm")])
whisker_hover_tooltips = HoverTool(names=['Whiskers'],tooltips=[("Dia", "@y mm")])
multiline_hover_tooltips = HoverTool(names=['LinePlot'],tooltips=[("Dia", "$y mm")])

f.add_tools(dot_hover_tooltips,whisker_hover_tooltips,multiline_hover_tooltips)

f.xaxis.axis_label = 'Head of Versions'
f.yaxis.axis_label = 'Diameter Predictions (mm)'

f.multi_line(y_xs, ys_std, color='blue', line_width = 3, legend = 'Mean Prediction +/- Std.Dev')
f.multi_line(y_xs, y_ys, color='deepskyblue',name='LinePlot',legend = 'Mean Prediction +/- Min & Max Value')
f.diamond(HoV_test_data, y_true, color='red', size=12, line_alpha=0,name='DiaPoints', legend = 'True Dia')
f.circle(HoV_test_data, y_means, color='blue', size=9, line_alpha=0,name='DiaPoints', legend = 'Predicted Dia (Mean)')

# whiskers (almost-0 height rects simpler than segments)
for i in range(len(y_xs)):
    f.rect(y_xs[i], y_ys[i], 0.2, 0.001, line_color="deepskyblue",name='Whiskers')

f.y_range=Range1d(135, 175)
f.legend.location = "top_center"
f.legend.click_policy="hide"

show(f)

In [25]:
output_file('plots/BNN_MCDropOut_v1_FDDN_LTR_Predictions.html', mode='inline')